在这个教程中，我想用taxi gps数据计算每一辆出租车在一天内的收入。如果能够成功计算，再看看高收入的出租车司机的行为，就可以分析出来：  
毕业以后要是失业了，怎么样开出租车才能赚更多的钱！升职加薪赢取白富美就在明天！

提供的基础数据是：
<div class="alert alert-info"><h2>提供的基础数据是：</h2><p>   
    数据：<br>  
    1.出租车原始GPS数据<br>  

 </p></div>
 
 <div class="alert alert-info"><h2>深圳出租车计价规则：</h2><p>   
(一)起步价：首2公里11.00元;<br>  
(二)里程价：超过2公里部分，每公里2.40元;<br>  
(三)返空费：每天的23时至次日凌晨6时，超过25公里部分，每公里按上述里程价的30%加收返空费：<br>  
(四)夜间附加费：夜间起步价16元，每天的23时至次日凌晨6时，按上述起步价和里程价的20%加收夜间附加费;<br>  
(五)候时费：每分钟0.80元;<br>  
(六)大件行李费;体积超过0.2立方米、重量超过20公斤的大件行李，每件0.50元。<br>  
 </p></div>  
 
 
在计算的时候，我们可以以订单开始的时间判定夜间还是日间。另外，规则（六）也考虑不了

# 思路
## 首先，我们的输入和输出是什么？

输入很简单，TAXIGPS数据  
输出的形式，应该是：每条订单，收入  
为了后续的分析，在输出的基础上，再增加：订单号，收入，车牌号，订单开始时间，订单结束时间，订单上车坐标，订单下车坐标，行驶里程


那么，从输入怎么变成输出？计算出租车费，我们需要几个东西：订单号，行驶里程，订单开始时间（判断是否为夜间），候时的时长

## 任务分解

搞清楚输入和输出后，应用我们结构化数据的存储及处理的思维方式，就可以进行任务分解了，我们要解决的就是以下几个任务:  

    任务1：由最原始的输入，数据清洗，输出表1，字段与原始数据一致  
    
    任务2：表1输入，整理出每个订单的轨迹，用来计算里程和候时时长，输出表2，字段为：订单号，轨迹点经纬度，轨迹点的时间（订单号这一列需要我们自己定义）,速度（用来计算候时时长）  
    
    任务3：表2输入，计算每个订单的里程和候时时长，输出表3，字段为：订单号，订单的里程，候时时长  
    
    任务4：表1输入，整理出每个订单的其他信息，输出表4，字段为：订单号，车牌号，订单开始时间，订单结束时间，订单上车坐标，订单下车坐标  
    
    任务5：表3与表4输入，连接两个表，再计算订单收入，输出表5，字段为：订单号，收入，车牌号，订单开始时间，订单结束时间，订单上车坐标，订单下车坐标，行驶里程

# 数据

In [2]:
import pandas as pd
#读取数据
data2 = pd.read_csv(r'data-sample/TaxiData-Sample',header = None)
#给数据命名列
data2.columns = ['VehicleNum', 'Stime', 'Lng', 'Lat', 'OpenStatus', 'Speed']
data2.head(5)

,VehicleNum,Stime,Lng,Lat,OpenStatus,Speed
0,22271,22:54:04,114.167000,22.718399,0,0
1,22271,18:26:26,114.190598,22.647800,0,4
2,22271,18:35:18,114.201401,22.649700,0,0
3,22271,16:02:46,114.233498,22.725901,0,24
4,22271,21:41:17,114.233597,22.720900,0,19


# 订单收入计算

## 数据清洗

In [2]:
data2=data2.sort_values(by=['VehicleNum','Sdate','Stime'])
data2 = data2[-((data2['OpenStatus'].shift(1)== data2['OpenStatus'].shift(-1))&\
      (data2['OpenStatus'].shift(1)!= data2['OpenStatus'])&\
      (data2['VehicleNum'].shift(1)==data2['VehicleNum'].shift(-1))&\
      (data2['VehicleNum']==data2['VehicleNum'].shift(1)))]
len(data2)

49571674

## 订单号生成

如果StatusChange为这一条数据的OpenStatus减去上一条数据的OpenStatus，这样就会出现：

|OpenStatus     |   OpenStatus1    |  StatusChange||
| ----------- |-----------|||
|0          |       0    |             0||
|0          |       0    |             0||
|0         |        0    |             0||
|0          |       0    |             0 |    ←此时乘客上车了|
|1          |       0    |             1  |   ←此时乘客上车了|
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0|    ←此时乘客下车了|
|0          |       1    |             -1  |  ←此时乘客下车了|
|0          |       0    |             0||
|0          |       0    |             0||
|0          |       0    |             0||


那么如果我只把所有OpenStatus==1的数据拿出来，就会有：


|OpenStatus     |   orderid(订单号：对StatusChange的累加，可以用pandas的.cumsum()功能)    |  StatusChange||
| ----------- |-----------|||
|1          |       0    |             0||
|1          |       0    |             0||
|1         |        0    |             0||
|1          |       0    |             0 ||
|1          |       1    |             1  |   ←新订单|
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       2   |             1  |  ←新订单|
|1          |       2    |             0||
|1          |       2    |             0||
|1          |       2    |             0||


In [3]:
data2['OpenStatus1'] = data2['OpenStatus'].shift()
data2['VehicleNum1'] = data2['VehicleNum'].shift()

data2['Lng1'] = data2['Lng'].shift()
data2['Lat1'] = data2['Lat'].shift()
data2['Sdate1'] = data2['Sdate'].shift()
data2['Stime1'] = data2['Stime'].shift()

data2['StatusChange'] = data2['OpenStatus1']-data2['OpenStatus']
data2 = data2[data2['OpenStatus1']==1]
data2['orderid'] = data2['StatusChange'].cumsum()
data2.head(50)

,Sdate,Stime,CompanyId,VehicleNum,Lng,Lat,Speed,Angle,OpenStatus,Usable,OpenStatus1,VehicleNum1,Lng1,Lat1,Sdate1,Stime1,StatusChange,orderid
10273111,20161219,1916,H,粤B002ZB,113.848099,22.606400,5.0,0,1,1,1.0,粤B002ZB,113.848099,22.606600,20161219.0,1901.0,0.0,0.0
10289874,20161219,1945,H,粤B002ZB,113.846901,22.605600,42.0,0,1,1,1.0,粤B002ZB,113.848099,22.606400,20161219.0,1916.0,0.0,0.0
16051679,20161219,2016,H,粤B002ZB,113.843292,22.604200,38.0,0,1,1,1.0,粤B002ZB,113.846901,22.605600,20161219.0,1945.0,0.0,0.0
16060663,20161219,2031,H,粤B002ZB,113.842201,22.604000,24.0,256,1,1,1.0,粤B002ZB,113.843292,22.604200,20161219.0,2016.0,0.0,0.0
16081035,20161219,2101,H,粤B002ZB,113.840797,22.606400,18.0,256,1,1,1.0,粤B002ZB,113.842201,22.604000,20161219.0,2031.0,0.0,0.0
16090481,20161219,2116,H,粤B002ZB,113.840599,22.606800,0.0,0,1,1,1.0,粤B002ZB,113.840797,22.606400,20161219.0,2101.0,0.0,0.0
7806,20161219,2202,H,粤B002ZB,113.840897,22.606001,46.0,0,1,1,1.0,粤B002ZB,113.840599,22.606800,20161219.0,2116.0,0.0,0.0
16503,20161219,2217,H,粤B002ZB,113.841797,22.604099,64.0,0,1,1,1.0,粤B002ZB,113.840897,22.606001,20161219.0,2202.0,0.0,0.0
25743,20161219,2232,H,粤B002ZB,113.843002,22.601700,61.0,0,1,1,1.0,粤B002ZB,113.841797,22.604099,20161219.0,2217.0,0.0,0.0
35386,20161219,2247,H,粤B002ZB,113.843697,22.600000,16.0,0,1,1,1.0,粤B002ZB,113.843002,22.601700,20161219.0,2232.0,0.0,0.0


In [6]:
#表2入手
table2 = data2[['orderid','Lng','Lat','Sdate','Stime','Speed']]
table2.head(5)

,orderid,Lng,Lat,Sdate,Stime,Speed
10273111,0.0,113.848099,22.6064,20161219,1916,5.0
10289874,0.0,113.846901,22.6056,20161219,1945,42.0
16051679,0.0,113.843292,22.6042,20161219,2016,38.0
16060663,0.0,113.842201,22.6040,20161219,2031,24.0
16081035,0.0,113.840797,22.6064,20161219,2101,18.0


## 订单里程计算与候车时间

In [13]:
#表2
#计算里程

#定义计算路径长度函数

from math import radians, cos, sin, asin, sqrt
 
def getdistance(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 =float(lon1), float(lat1), float(lon2), float(lat2)
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r * 1000




In [36]:
#生成订单的里程

table3 = table2.copy()
table3['Lng1'] = table3['Lng'].shift(-1)
table3['Lat1'] = table3['Lat'].shift(-1)
table3['orderid1'] = table3['orderid'].shift(-1)
table3['Sdate1'] = table3['Sdate'].shift(-1)
table3['Stime1'] = table3['Stime'].shift(-1)

table3 = table3[(table3['orderid1'] == table3['orderid'])]
#计算每个点与下一个点的距离
table3['distance'] = table3.apply(lambda row:getdistance(row['Lng'],row['Lat'],row['Lng1'],row['Lat1']),axis = 1)


#计算每个点与下一个点的时间差
tmp = (1000000+table3['Stime']).astype('str').apply(lambda row:3600*int(row[1:3])+60*int(row[3:5])+int(row[5:7]))
tmp2 = (1000000+table3['Stime1']).astype('str').apply(lambda row:3600*int(row[1:3])+60*int(row[3:5])+int(row[5:7]))
table3['interval'] = tmp2-tmp

#集计得到出租车路径长度
orderlenth = table3[['orderid','distance']].groupby('orderid').sum().reset_index()

waittime = table3[table3['distance']==0][['orderid','interval']].groupby('orderid').sum()


table3 = pd.merge(orderlenth,waittime,on = 'orderid',how='left')
table3.head(10)

,orderid,distance,interval
0,0.0,4263.463445,141.0
1,1.0,6705.718061,64.0
2,2.0,17723.652287,273.0
3,3.0,5379.988918,46.0
4,4.0,6601.635257,45.0
5,5.0,3056.065243,15.0
6,6.0,2731.186625,20.0
7,7.0,17279.247734,94.0
8,8.0,12019.105999,45.0
9,9.0,6689.860822,46.0


In [38]:
#存储结果
table3.fillna(0).to_csv(r'/MD3800I/hadoop/data/TAXIGPS/余庆/order_dis_int.csv',index = None)

In [39]:
table3_1.fillna(0)

,orderid,distance,interval
0,0.0,4263.463445,141.0
1,1.0,6705.718061,64.0
2,2.0,17723.652287,273.0
3,3.0,5379.988918,46.0
4,4.0,6601.635257,45.0
5,5.0,3056.065243,15.0
6,6.0,2731.186625,20.0
7,7.0,17279.247734,94.0
8,8.0,12019.105999,45.0
9,9.0,6689.860822,46.0


## 匹配表3与表4

In [75]:
#表4的字段
#订单号，车牌号，订单开始时间，订单结束时间，订单上车坐标，订单下车坐标  

In [57]:
o = data2.iloc[:1].append(data2[data2['StatusChange']==1])
d = data2[(data2['StatusChange']==1).shift(-1).fillna(False)]
table4 = o.append(d).sort_values(by = ['orderid','Stime'])[['orderid','VehicleNum','Sdate','Stime','Lng','Lat']]
table4

,orderid,VehicleNum,Sdate,Stime,Lng,Lat
10273111,0.0,粤B002ZB,20161219,1916,113.848099,22.606400
26565566,0.0,粤B002ZB,20161219,2750,113.859299,22.586000
26577712,1.0,粤B002ZB,20161219,2810,113.859299,22.586000
28447160,1.0,粤B002ZB,20161219,14137,113.871201,22.574400
38192191,2.0,粤B002ZB,20161219,14209,113.873299,22.573200
15908899,2.0,粤B002ZB,20161219,23221,113.915398,22.543501
15928358,3.0,粤B002ZB,20161219,23251,113.911797,22.545099
21575266,3.0,粤B002ZB,20161219,31211,113.892899,22.585199
21605472,4.0,粤B002ZB,20161219,31301,113.891899,22.584801
22226928,4.0,粤B002ZB,20161219,33009,113.901108,22.568199


In [60]:
table4['isd'] = [i%2 for i in range(len(table4))]
table4.head(5)

,orderid,VehicleNum,Sdate,Stime,Lng,Lat,isd
10273111,0.0,粤B002ZB,20161219,1916,113.848099,22.6064,0
26565566,0.0,粤B002ZB,20161219,2750,113.859299,22.5860,1
26577712,1.0,粤B002ZB,20161219,2810,113.859299,22.5860,0
28447160,1.0,粤B002ZB,20161219,14137,113.871201,22.5744,1
38192191,2.0,粤B002ZB,20161219,14209,113.873299,22.5732,0


In [65]:
table4['Edate'] = table4['Sdate'].shift(-1)
table4['Etime'] = table4['Stime'].shift(-1)
table4['ELng'] = table4['Lng'].shift(-1)
table4['ELat'] = table4['Lat'].shift(-1)
table4 = table4[(table4['isd']==0)&(-table4['Edate'].isnull())]

In [68]:
table3 = table3_1.fillna(0)

In [69]:
table5 = pd.merge(table4,table3,on = 'orderid',how = 'left')

In [73]:
#每天的23时至次日凌晨6时为夜间
table5['isnight'] = (table5['Stime']<60000)|(table5['Stime']>230000)

## 收入计算

(一)起步价：首2公里11.00元;  
(二)里程价：超过2公里部分，每公里2.40元;  
(三)返空费：每天的23时至次日凌晨6时，超过25公里部分，每公里按上述里程价的30%加收返空费：  
(四)夜间附加费：夜间起步价16元，每天的23时至次日凌晨6时，按上述起步价和里程价的20%加收夜间附加费;  
(五)候时费：每分钟0.80元;  

In [82]:
#起步价
table5['isnight']*(16-11)+11

0         16
1         16
2         16
3         16
4         16
5         16
6         16
7         16
8         16
9         11
10        11
11        11
12        11
13        11
14        11
15        11
16        11
17        11
18        11
19        11
20        11
21        11
22        11
23        11
24        11
25        11
26        11
27        11
28        11
29        11
          ..
276563    16
276564    16
276565    16
276566    16
276567    16
276568    11
276569    11
276570    11
276571    11
276572    11
276573    11
276574    11
276575    11
276576    11
276577    11
276578    11
276579    11
276580    11
276581    11
276582    11
276583    11
276584    11
276585    11
276586    11
276587    11
276588    11
276589    11
276590    11
276591    11
276592    16
Name: isnight, Length: 276593, dtype: int64

In [80]:
#里程价
((table5['distance']-2000)>0)*(table5['distance']-2000)*2.4/1000

0          5.432312
1         11.293723
2         37.736765
3          8.111973
4         11.043925
5          2.534557
6          1.754848
7         36.670195
8         24.045854
9         11.255666
10         3.416254
11        14.577239
12         2.082086
13        33.176878
14        35.476060
15         5.325484
16         7.687273
17         2.086493
18         5.721425
19         7.926922
20         7.578569
21         7.674420
22        -0.000000
23         4.972708
24         4.152223
25         5.356027
26         5.148268
27        -0.000000
28        23.605340
29        15.019851
            ...    
276563     3.594010
276564    26.796817
276565     3.722326
276566    10.690023
276567    49.638823
276568    -0.000000
276569    13.861234
276570    24.638198
276571    71.339469
276572    66.005459
276573    39.108138
276574     8.428137
276575    34.548883
276576     8.493268
276577    17.123160
276578    16.793498
276579    10.522967
276580     0.027979
276581     9.672369


In [84]:
#返空费
table5['isnight']*((table5['distance']-25000)>0)*((table5['distance']-25000)*2.4*0.3/1000)

In [86]:
#夜间附加费(里程价)
table5['isnight']*((table5['distance']-2000)>0)*(table5['distance']-2000)*2.4*0.2/1000

0         1.086462
1         2.258745
2         7.547353
3         1.622395
4         2.208785
5         0.506911
6         0.350970
7         7.334039
8         4.809171
9         0.000000
10        0.000000
11        0.000000
12        0.000000
13        0.000000
14        0.000000
15        0.000000
16        0.000000
17        0.000000
18        0.000000
19        0.000000
20        0.000000
21        0.000000
22       -0.000000
23        0.000000
24        0.000000
25        0.000000
26        0.000000
27       -0.000000
28        0.000000
29        0.000000
            ...   
276563    0.718802
276564    5.359363
276565    0.744465
276566    2.138005
276567    9.927765
276568   -0.000000
276569    0.000000
276570    0.000000
276571    0.000000
276572    0.000000
276573    0.000000
276574    0.000000
276575    0.000000
276576    0.000000
276577    0.000000
276578    0.000000
276579    0.000000
276580    0.000000
276581    0.000000
276582    0.000000
276583    0.000000
276584    0.

In [88]:
#候时费
table5['interval']/60*0.8

0          1.880000
1          0.853333
2          3.640000
3          0.613333
4          0.600000
5          0.200000
6          0.266667
7          1.253333
8          0.600000
9          0.613333
10         0.000000
11         3.626667
12         0.626667
13         3.026667
14         2.026667
15         1.013333
16         1.000000
17         1.400000
18         0.200000
19         1.000000
20         0.400000
21         1.413333
22         0.200000
23         1.200000
24         0.813333
25         2.013333
26         1.813333
27         0.813333
28         0.400000
29         1.000000
            ...    
276563     0.600000
276564     1.200000
276565     0.160000
276566     2.000000
276567     2.800000
276568     0.000000
276569     0.400000
276570     0.400000
276571     1.400000
276572     3.000000
276573     1.160000
276574     0.400000
276575     6.813333
276576     3.200000
276577     5.013333
276578     7.000000
276579     5.200000
276580     5.800000
276581     5.066667


In [90]:
#起步价
table5['price'] = \
table5['isnight']*(16-11)+11+\
((table5['distance']-2000)>0)*(table5['distance']-2000)*2.4/1000+\
table5['isnight']*((table5['distance']-25000)>0)*((table5['distance']-25000)*2.4*0.3/1000)+\
table5['isnight']*((table5['distance']-2000)>0)*(table5['distance']-2000)*2.4*0.2/1000+\
table5['interval']/60*0.8
table5

,orderid,VehicleNum,Sdate,Stime,Lng,Lat,isd,Edate,Etime,ELng,ELat,distance,interval,isnight,price
0,0.0,粤B002ZB,20161219,1916,113.848099,22.606400,0,20161219.0,2750.0,113.859299,22.586000,4263.463445,141.0,True,24.398775
1,1.0,粤B002ZB,20161219,2810,113.859299,22.586000,0,20161219.0,14137.0,113.871201,22.574400,6705.718061,64.0,True,30.405801
2,2.0,粤B002ZB,20161219,14209,113.873299,22.573200,0,20161219.0,23221.0,113.915398,22.543501,17723.652287,273.0,True,64.924119
3,3.0,粤B002ZB,20161219,23251,113.911797,22.545099,0,20161219.0,31211.0,113.892899,22.585199,5379.988918,46.0,True,26.347701
4,4.0,粤B002ZB,20161219,31301,113.891899,22.584801,0,20161219.0,33009.0,113.901108,22.568199,6601.635257,45.0,True,29.852710
5,5.0,粤B002ZB,20161219,33030,113.900703,22.567900,0,20161219.0,34530.0,113.882202,22.589100,3056.065243,15.0,True,19.241468
6,6.0,粤B002ZB,20161219,34552,113.882004,22.589199,0,20161219.0,35812.0,113.897903,22.577000,2731.186625,20.0,True,18.372484
7,7.0,粤B002ZB,20161219,35814,113.890800,22.587000,0,20161219.0,50150.0,113.808296,22.626900,17279.247734,94.0,True,61.257567
8,8.0,粤B002ZB,20161219,50210,113.806297,22.625900,0,20161219.0,63414.0,113.898598,22.571301,12019.105999,45.0,True,45.455025
9,9.0,粤B002ZB,20161219,63440,113.899902,22.571400,0,20161219.0,70808.0,113.878601,22.571301,6689.860822,46.0,False,22.868999
